In [23]:
import os

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
from src.chromaDB_setup import *
from src.github_parser import *
from src.embedding import *
import chromadb
from chromadb.config import Settings
import numpy as np

In [25]:
repo = g.get_repo("vl4duu/vlad-mutilica.space")
# ("khoj-ai/khoj")

In [26]:
repo_files = get_repo_files("vl4duu/vlad-mutilica.space")

In [27]:
chunked_docs = chunk_repository_files(repo_files)

textual_embeddings = np.array([embed_textual_metadata(doc["content"]) for doc in chunked_docs['textual_chunks']])

# Process code chunks with CodeBERT
code_embeddings = np.array([generate_code_embedding(doc["content"]) for doc in chunked_docs['code_chunks']])

embedded_chunks = {
    'textual_embeddings': np.array(textual_embeddings),
    'code_embeddings': np.array(code_embeddings)
}

print(chunked_docs.keys())

dict_keys(['textual_chunks', 'code_chunks'])


In [28]:
chrombaDb_collections = setup_chroma_collections(chunked_docs, embedded_chunks)

Add of existing embedding ID: README.md_0
Insert of existing embedding ID: README.md_0
Insert of existing embedding ID: src/views/index/animations.js_0
Add of existing embedding ID: src/views/index/animations.js_0
Insert of existing embedding ID: src/views/index/index.html_1
Add of existing embedding ID: src/views/index/index.html_1
Insert of existing embedding ID: src/views/portfolio/portfolio.html_2
Add of existing embedding ID: src/views/portfolio/portfolio.html_2
Insert of existing embedding ID: src/views/portfolio/portfolio.js_3
Add of existing embedding ID: src/views/portfolio/portfolio.js_3


In [34]:
def retrieve_relevant_chunks(query, top_k=3):
    # Generate query embedding
    query_embedding = embed_textual_metadata(query)
    print(query_embedding)
    # Query ChromaDB
    results = chrombaDb_collections['textual_collection'].query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    # Return relevant documents and metadata
    return results["documents"], results["metadatas"]

In [35]:
query = "What is this repo supposed to do?"
retrieve_relevant_chunks(query)

Number of requested results 3 is greater than number of elements in index 1, updating n_results = 1


[-0.0022076242603361607, -0.003987637814134359, 0.017470944672822952, 0.007547665387392044, -0.030543608590960503, -0.002112599555402994, -0.027272049337625504, 0.0069978805258870125, -0.00324271316640079, -0.0015051212394610047, 0.021095452830195427, -0.013330589048564434, -0.028453068807721138, 0.00042209570528939366, -0.008178899995982647, -0.00404193764552474, 0.012998002581298351, -0.04061621427536011, 0.0032155632507056, -0.0069639431312680244, -0.007371190935373306, -0.008077087812125683, -0.02560233138501644, -0.002614872297272086, -0.018312590196728706, 0.007486578077077866, 0.018285440281033516, -0.025045759975910187, 0.002292467514052987, 0.00869474746286869, 0.03415454179048538, -0.01998230814933777, -0.020511731505393982, -0.006370039656758308, 0.0010800559539347887, -0.013588512316346169, 0.01671074889600277, 0.002986486302688718, -0.010751350782811642, 0.003473487216979265, 0.01819041557610035, -0.0016875343862921, -0.009047695435583591, -0.00017329255933873355, -0.01054

([['# vlad-mutilica.space <h4>personal website designed to serve as a carte de visite</h3>']],
 [[{'content_type': 'text', 'file_name': 'README.md'}]])